In [1]:
# Import Modules
import os
import plaid
import requests
import datetime
import pendulum
import json
import pandas as pd
%matplotlib inline

In [2]:
# Define JSON prettifier parameters
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [3]:
# Read API keys in from txt files
file1 = open('./keys/plaid_client_id.txt','r')
plaid_client_id = file1.readline()
file1.close()

file2 = open('./keys/plaid_secret.txt','r')
plaid_secret = file2.readline()
file2.close()

file3 = open('./keys/plaid_public_key.txt','r')
plaid_public_key = file3.readline()
file3.close()

In [4]:
# Set Plaid environment variables
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID', plaid_client_id)
PLAID_SECRET = os.getenv('PLAID_SECRET', plaid_secret)
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY', plaid_public_key)
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [5]:
# Set Plaid Institition ID
INSTITUTION_ID = "ins_109508"

In [6]:
# Set Plaid development environment
client = plaid.Client(client_id = PLAID_CLIENT_ID, 
                      secret=PLAID_SECRET,
                      public_key=PLAID_PUBLIC_KEY, 
                      environment=PLAID_ENV, 
                      api_version='2019-05-29')

### 2. Generate a public token

In [7]:
# create response from sandbox - detailed in plaid api documentation
create_response = client.Sandbox.public_token.create(
    INSTITUTION_ID, ['transactions'])

### 3. Exchange the public token for an access token

In [8]:
# Set access token from exchange response
response = client.Item.public_token.exchange(create_response['public_token'])
access_token = response['access_token']

### 4. Fetch Accounts

In [9]:
# Create call response for Accounts
accounts_response = client.Accounts.get(access_token)

In [10]:
# Fetch Accounts
accounts = accounts_response['accounts']
pretty_print_response(accounts[:3])

[
    {
        "account_id": "AenmbA87w7H1pzvz5Q6AsNDPm8677EC1NGNx4",
        "balances": {
            "available": 100,
            "current": 110,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "0000",
        "name": "Plaid Checking",
        "official_name": "Plaid Gold Standard 0% Interest Checking",
        "subtype": "checking",
        "type": "depository"
    },
    {
        "account_id": "GpQmB9vLnLUN1rGrzRZMH546d8mPPqf1gEg7A",
        "balances": {
            "available": 200,
            "current": 210,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "1111",
        "name": "Plaid Saving",
        "official_name": "Plaid Silver Standard 0.1% Interest Saving",
        "subtype": "savings",
        "type": "depository"
    },
    {
        "account_id": "nK5j1laLRLsx95j5Xrg8IK1vwGyEEDu6qxqMG"

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [ ]:
# YOUR CODE HERE

### 2. Print the categories for each transaction

In [ ]:
# YOUR CODE HERE

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [ ]:
# YOUR CODE HERE

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
# YOUR CODE HERE

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# YOUR CODE HERE

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE

### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE